In [2]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
from sklearn.datasets import make_regression
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import statsmodels.api as sm
import matplotlib
import itertools
from pylab import rcParams
import pickle
import warnings
warnings.filterwarnings("ignore")

In [18]:
mapping = {}
for i in range(1,7):
    file_path = 'low_freq/house_' + str(i) + '/labels.dat'
    mapping[i] = {}
    with open(file_path) as file:
        for line in file:
            mapping[i][int(line.split(' ')[0].strip())] = line.split(' ')[1].strip() + '_' + line.split(' ')[0].strip()
            

'bathroom_gfi_7'

In [31]:
df = {}
for i in range(1,7):
    file_path = 'low_freq/house_' + str(i) + '/channel_' + str(i) + '.dat'
    df[i] = pd.read_csv(file_path , sep = ' ', names = ['TimeStamp', mapping[i][1]])
    for j in range(2, len(mapping[i])+1):
        curr_file_path =  'low_freq/house_' + str(i) + '/channel_' + str(j) + '.dat'
        curr = pd.read_csv(curr_file_path , sep = ' ', names = ['TimeStamp', mapping[i][j]])
        df[i] = pd.merge(df[i], curr, on='TimeStamp')
    display(df[i])

,TimeStamp,mains_1,mains_2,oven_3,oven_4,refrigerator_5,dishwaser_6,kitchen_outlets_7,kitchen_outlets_8,lighting_9,...,microwave_11,bathroom_gfi_12,electric_heat_13,stove_14,kitchen_outlets_15,kitchen_outlets_16,lighting_17,lighting_18,washer_dryer_19,washer_dryer_20
0,1303132933,222.20,118.83,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
1,1303132936,223.17,119.19,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
2,1303132940,223.60,118.92,0.0,0.0,6.0,0.0,34.0,22.0,81.0,...,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
3,1303132943,222.91,119.16,0.0,0.0,6.0,1.0,35.0,21.0,81.0,...,5.0,1.0,0.0,0.0,1.0,0.0,65.0,46.0,0.0,0.0
4,1303132946,222.94,118.83,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
5,1303132950,222.90,118.76,0.0,0.0,6.0,0.0,33.0,21.0,81.0,...,5.0,1.0,0.0,0.0,1.0,0.0,66.0,46.0,0.0,0.0
6,1303132953,222.96,118.88,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,5.0,1.0,0.0,0.0,1.0,0.0,65.0,46.0,0.0,0.0
7,1303132957,222.54,118.36,0.0,0.0,6.0,0.0,34.0,21.0,82.0,...,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
8,1303132960,226.03,119.17,0.0,0.0,6.0,0.0,38.0,21.0,82.0,...,5.0,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0
9,1303132964,222.96,119.03,0.0,0.0,6.0,0.0,34.0,21.0,81.0,...,5.0,1.0,0.0,0.0,1.0,0.0,65.0,46.0,0.0,0.0


,TimeStamp,mains_1,mains_2,kitchen_outlets_3,lighting_4,stove_5,microwave_6,washer_dryer_7,kitchen_outlets_8,refrigerator_9,dishwaser_10,disposal_11
0,1303104700,22.61,22.61,1.0,8.0,1.0,4.0,4.0,4.0,6.0,1.0,0.0
1,1303104704,22.61,22.61,0.0,8.0,1.0,5.0,5.0,4.0,6.0,0.0,0.0
2,1303104707,22.61,22.61,1.0,8.0,1.0,4.0,4.0,4.0,6.0,1.0,0.0
3,1303104710,22.57,22.57,0.0,8.0,1.0,4.0,4.0,3.0,7.0,0.0,0.0
4,1303104725,22.61,22.61,1.0,8.0,1.0,4.0,4.0,3.0,7.0,1.0,0.0
5,1303104739,22.59,22.59,0.0,8.0,0.0,4.0,5.0,3.0,7.0,0.0,0.0
6,1303104743,22.62,22.62,0.0,8.0,0.0,4.0,5.0,3.0,6.0,0.0,0.0
7,1303104746,22.58,22.58,0.0,8.0,1.0,4.0,5.0,4.0,6.0,0.0,0.0
8,1303104749,22.56,22.56,1.0,8.0,1.0,4.0,4.0,3.0,6.0,1.0,0.0
9,1303104753,22.62,22.62,1.0,8.0,1.0,5.0,5.0,3.0,6.0,0.0,0.0


,TimeStamp,mains_1,mains_2,outlets_unknown_3,outlets_unknown_4,lighting_5,electronics_6,refrigerator_7,disposal_8,dishwaser_9,...,washer_dryer_13,washer_dryer_14,lighting_15,microwave_16,lighting_17,smoke_alarms_18,lighting_19,bathroom_gfi_20,kitchen_outlets_21,kitchen_outlets_22
0,1302930690,2.0,577.30,2.0,0.0,1.0,126.0,119.0,0.0,0.0,...,0.0,0.0,1.0,2.0,195.0,2.0,23.0,0.0,0.0,0.0
1,1302930693,1.0,577.39,1.0,0.0,1.0,125.0,118.0,0.0,1.0,...,0.0,0.0,1.0,2.0,195.0,2.0,23.0,0.0,0.0,0.0
2,1302930696,1.0,577.88,1.0,0.0,1.0,124.0,119.0,0.0,0.0,...,0.0,0.0,1.0,2.0,195.0,2.0,24.0,0.0,0.0,0.0
3,1302930700,2.0,578.44,2.0,0.0,1.0,127.0,119.0,0.0,1.0,...,0.0,0.0,1.0,2.0,195.0,1.0,23.0,0.0,0.0,0.0
4,1302930703,1.0,575.92,1.0,0.0,1.0,127.0,117.0,0.0,0.0,...,0.0,0.0,1.0,2.0,195.0,2.0,23.0,0.0,0.0,0.0
5,1302930707,2.0,577.49,2.0,0.0,1.0,127.0,119.0,0.0,1.0,...,0.0,0.0,1.0,2.0,195.0,2.0,23.0,0.0,0.0,0.0
6,1302930710,2.0,578.95,2.0,0.0,1.0,125.0,120.0,0.0,0.0,...,0.0,0.0,1.0,2.0,195.0,1.0,23.0,0.0,0.0,0.0
7,1302930714,1.0,577.99,1.0,0.0,1.0,127.0,118.0,0.0,0.0,...,0.0,0.0,1.0,2.0,195.0,1.0,24.0,0.0,0.0,0.0
8,1302930717,2.0,578.72,2.0,0.0,1.0,127.0,119.0,0.0,0.0,...,0.0,0.0,1.0,2.0,195.0,2.0,24.0,0.0,0.0,0.0
9,1302930721,1.0,578.39,1.0,0.0,1.0,124.0,119.0,0.0,0.0,...,0.0,0.0,1.0,2.0,195.0,1.0,23.0,0.0,0.0,0.0


,TimeStamp,mains_1,mains_2,lighting_3,furance_4,kitchen_outlets_5,outlets_unknown_6,washer_dryer_7,stove_8,air_conditioning_9,...,miscellaeneous_11,smoke_alarms_12,lighting_13,kitchen_outlets_14,dishwaser_15,bathroom_gfi_16,bathroom_gfi_17,lighting_18,lighting_19,air_conditioning_20
0,1303002992,4.0,86.58,243.0,4.0,1.0,0.0,0.0,5.0,0.0,...,2.0,0.0,51.0,139.0,0.0,0.0,0.0,15.0,0.0,0.0
1,1303002995,4.0,86.56,244.0,4.0,0.0,0.0,0.0,5.0,0.0,...,1.0,0.0,51.0,138.0,0.0,0.0,0.0,15.0,0.0,0.0
2,1303002998,5.0,86.62,243.0,5.0,1.0,0.0,1.0,5.0,0.0,...,2.0,0.0,51.0,140.0,0.0,0.0,0.0,15.0,0.0,0.0
3,1303003002,4.0,86.50,241.0,4.0,1.0,0.0,0.0,5.0,0.0,...,2.0,0.0,51.0,139.0,0.0,0.0,0.0,14.0,0.0,0.0
4,1303003005,4.0,86.61,241.0,4.0,1.0,0.0,0.0,5.0,0.0,...,2.0,1.0,51.0,137.0,0.0,0.0,0.0,15.0,0.0,0.0
5,1303003009,4.0,86.50,242.0,4.0,1.0,0.0,0.0,5.0,0.0,...,2.0,1.0,50.0,141.0,0.0,0.0,0.0,15.0,0.0,0.0
6,1303003012,5.0,86.70,242.0,5.0,1.0,0.0,0.0,5.0,0.0,...,2.0,1.0,50.0,140.0,0.0,0.0,0.0,14.0,0.0,0.0
7,1303003016,5.0,86.73,241.0,5.0,1.0,0.0,0.0,5.0,0.0,...,2.0,0.0,50.0,139.0,0.0,0.0,0.0,15.0,0.0,2.5
8,1303003019,5.0,86.55,242.0,5.0,1.0,0.0,0.0,5.0,0.0,...,1.0,1.0,51.0,139.0,0.0,0.0,0.0,15.0,0.0,0.0
9,1303003023,4.0,86.79,242.0,4.0,1.0,0.0,0.0,5.0,0.0,...,1.0,0.0,51.0,142.0,0.0,0.0,0.0,14.0,0.0,0.0


,TimeStamp,mains_1,mains_2,microwave_3,lighting_4,outlets_unknown_5,furance_6,outlets_unknown_7,washer_dryer_8,washer_dryer_9,...,lighting_17,refrigerator_18,lighting_19,dishwaser_20,disposal_21,electronics_22,lighting_23,kitchen_outlets_24,kitchen_outlets_25,outdoor_outlets_26
0,1303100647,4.0,196.83,4.0,0.0,4.0,6.0,1.0,0.0,0.0,...,0.0,158.0,1.0,1.0,0.0,7.5,67.5,0.0,0.0,0.0
1,1303100651,4.0,197.87,4.0,0.0,4.0,6.0,1.0,0.0,0.0,...,0.0,160.0,1.0,1.0,0.0,7.5,67.5,0.0,0.0,0.0
2,1303100654,4.0,197.09,3.0,0.0,4.0,6.0,1.0,0.0,0.0,...,0.0,161.0,1.0,1.0,0.0,10.0,67.5,0.0,0.0,0.0
3,1303100658,4.0,196.08,4.0,0.0,4.0,6.0,0.0,0.0,0.0,...,0.0,160.0,1.0,2.0,0.0,7.5,67.5,0.0,0.0,0.0
4,1303100661,4.0,196.70,4.0,0.0,4.0,6.0,0.0,0.0,0.0,...,0.0,157.0,1.0,1.0,0.0,7.5,67.5,0.0,0.0,0.0
5,1303100664,4.0,195.78,3.0,0.0,4.0,6.0,0.0,0.0,0.0,...,0.0,160.0,1.0,1.0,0.0,10.0,67.5,0.0,0.0,0.0
6,1303100668,4.0,196.65,3.0,0.0,4.0,6.0,0.0,0.0,0.0,...,0.0,158.0,1.0,1.0,0.0,7.5,67.5,0.0,0.0,0.0
7,1303100671,4.0,196.32,4.0,0.0,4.0,6.0,1.0,0.0,0.0,...,0.0,161.0,1.0,1.0,0.0,10.0,67.5,0.0,0.0,0.0
8,1303100675,4.0,196.93,3.0,0.0,4.0,6.0,0.0,0.0,0.0,...,0.0,160.0,1.0,1.0,0.0,7.5,67.5,0.0,0.0,0.0
9,1303100678,4.0,196.96,3.0,0.0,4.0,6.0,0.0,0.0,0.0,...,0.0,158.0,1.0,1.0,0.0,7.5,67.5,0.0,0.0,0.0


,TimeStamp,mains_1,mains_2,kitchen_outlets_3,washer_dryer_4,stove_5,electronics_6,bathroom_gfi_7,refrigerator_8,dishwaser_9,outlets_unknown_10,outlets_unknown_11,electric_heat_12,kitchen_outlets_13,lighting_14,air_conditioning_15,air_conditioning_16,air_conditioning_17
0,1306006791,0.0,255.62,5.0,2.0,3.0,0.0,3.0,1.0,0.0,0.0,26.0,88.0,0.0,124.0,4.0,0.0,0.0
1,1306006797,0.0,257.48,5.0,3.0,2.0,0.0,3.0,0.0,0.0,0.0,27.0,94.0,1.0,116.0,4.0,0.0,0.0
2,1306006800,0.0,255.07,5.0,3.0,2.0,0.0,4.0,0.0,0.0,0.0,27.0,95.0,0.0,116.0,4.0,0.0,0.0
3,1306006804,0.0,255.78,5.0,2.0,3.0,0.0,3.0,0.0,0.0,0.0,27.0,94.0,1.0,116.0,4.0,0.0,0.0
4,1306006807,0.0,256.38,5.0,3.0,2.0,0.0,3.0,0.0,0.0,0.0,27.0,94.0,1.0,115.0,4.0,0.0,0.0
5,1306006813,0.0,262.64,5.0,3.0,3.0,0.0,3.0,0.0,0.0,0.0,26.0,95.0,1.0,122.0,4.0,0.0,0.0
6,1306006820,0.0,255.98,5.0,3.0,2.0,0.0,4.0,0.0,0.0,0.0,26.0,95.0,0.0,116.0,4.0,0.0,0.0
7,1306006827,0.0,255.83,5.0,3.0,2.0,0.0,3.0,0.0,0.0,0.0,26.0,93.0,1.0,116.0,4.0,0.0,0.0
8,1306006833,0.0,254.32,5.0,2.0,3.0,0.0,3.0,0.0,0.0,0.0,26.0,89.0,0.0,118.0,4.0,0.0,0.0
9,1306006842,0.0,255.10,5.0,3.0,2.0,0.0,3.0,0.0,0.0,0.0,26.0,95.0,0.0,116.0,4.0,0.0,0.0
